In [212]:
class Constraint:
    def __init__(self, vars):
        self.vars = vars

    def __call__(self, infoRt, assignments):
        pass

In [137]:
class CSP:
    def __init__(self):
        self.vars = []
        self.constraints = []
        self.information = {}

    def addVar(self,  key):
        self.vars.append(key)

    def setInformationRetriever(self, retriever):
        self.retriever = retriever

    def addConstraint(self, constraint):
        self.constraints.append(constraint)

In [18]:
import numpy as np
from sklearn.linear_model import LinearRegression

class CryptoIndicators:
    @classmethod
    def absolute_trend(cls, pC):
        pX = pC.copy()
        pX[0] = 0
        for i in range(len(pC) - 1):
            now = pC[i+1]
            prv = pC[i]
            if now > prv:
                pX[i+1] = 1
            elif now == prv:
                pX[i+1] = 0
            else:
                pX[i+1] = -1
        return pX
    
    @classmethod
    def correlative_compliance(cls, c1, c2):
        x1 = cls.absolute_trend(c1)
        x2 = cls.absolute_trend(c2)
        return np.corrcoef(x1, x2)[0][1]

    @classmethod
    def trendScore(cls, prices, lookback=50):
        r = LinearRegression()
        r.fit(np.array(range(len(prices)))[-lookback:].reshape(-1,1), prices[-lookback:])
        return r.coef_[0]

In [266]:
import joblib
import pandas as pd

class CryptoInfoRetriever:
    def __init__(self, framesPath, fcasPath, time=100, balance=1000):
        self.frames = joblib.load(framesPath)
        self.fcas = pd.read_csv(fcasPath)
        self.tokens = list(set(self.fcas.symbol.to_list()))
        self.tokenInfos = {}
        for t in self.tokens:
            self.tokenInfos[t] = {
                "shares": 0,
                "totalPayment": 0,
            }
        self.time = time 
        self.balance = balance
    
    def setTime(self, time):
        self.time = time
    
    def next(self):
        self.time += 1

    def finished(self):
        return self.time == 999

    def _build_pairs(self):
        pairAnalysis = []
        pairTracker = []
        for t in self.tokens:
            for t2 in self.tokens:
                if t != t2 and ((t + t2) not in pairTracker):
                    pairAnalysis.append({
                        "asset-1": t,
                        "asset-2": t2,
                        "quote-asset": "usdt"
                    })
                    pairTracker.append(t + t2)
                    pairTracker.append(t2 + t)
        cols = [k for k in pairAnalysis[0]]
        self.pairDf = pd.DataFrame(pairAnalysis, columns=cols)
    
    def _populate_corr(self):
        def corrCalc(d):
            a1 = d["asset-1"]
            a2 = d["asset-2"]
            tId = "{}usdt".format(a1.lower())
            t2Id = "{}usdt".format(a2.lower())
            c1 = self.frames[tId]["Close"].to_numpy()[:self.time]
            c2 = self.frames[t2Id]["Close"].to_numpy()[:self.time]

            corr = CryptoIndicators.correlative_compliance(c1, c2)

            d1 = self.tokenInfos[a1]
            d2 = self.tokenInfos[a2]
            if "corr-compliance" not in d1:
                d1["corr-compliance"] = {}
            if "corr-compliance" not in d2:
                d2["corr-compliance"] = {}
            d1["corr-compliance"][a2] = corr
            d2["corr-compliance"][a1] = corr
            
            return corr
        self.pairDf["corr-compliance"] = self.pairDf.apply(corrCalc, axis=1)
    
    def _populate_trends(self):
        for t in self.tokens:
            tId = "{}usdt".format(t.lower()) 
            c = self.frames[tId]["Close"].to_numpy()[:self.time]
            self.tokenInfos[t]["trend-score"] = CryptoIndicators.trendScore(c, 30)
    
    def _populate_fcas(self):
        for t in self.tokens:
            self.tokenInfos[t]["fcas"] = self.fcas[self.fcas.symbol == t].value.values[0]

    def _populate_close(self):
        for t in self.tokens:
            tId = "{}usdt".format(t.lower()) 
            c = self.frames[tId]["Close"].to_numpy()[:self.time]
            self.tokenInfos[t]["close"] = c[-1]

    def _populate_domain(self):
        for t in self.tokens:
            close = self.tokenInfos[t]["close"] 
            domain = []
            for i in range(71):
                holding = ((i) * self.balance) / 100
                domain.append(holding / close)
            self.tokenInfos[t]["domain"] = domain

    def build(self):
        if not hasattr(self, "pairDf") or self.pairDf is None:
            self._build_pairs()
        self._populate_corr()
        self._populate_trends()
        self._populate_fcas()
        self._populate_close()
        self._populate_domain()

    def order(self, symbol, count):
        s = symbol.upper()
        self.tokenInfos[s]["shares"] += count
        self.balance -= count * self.tokenInfos[s]["close"]
        self.tokenInfos[s]["totalPayment"] += count * self.tokenInfos[s]["close"]
    
    def getInfo(self, symbol):
        s = symbol.upper()
        return self.tokenInfos[s]

In [223]:
class FcasConstraint(Constraint):
    def __init__(self):
        super().__init__(None)
    
    def __call__(self, infoRt, assignments):
        if len(assignments) < 24:
            return True
        information = infoRt.tokenInfos
        vars = infoRt.tokens
        weightSum = 0
        weightedSum = 0
        for t in vars:
            weightedSum += information[t]["totalPayment"] * information[t]["fcas"]
            weightSum += information[t]["totalPayment"]
        weightedFcasMean = weightedSum / weightSum
        return weightedFcasMean > 800

In [270]:
class BalanceHoldingsConstraint(Constraint):
    def __init__(self):
        super().__init__(None)
    
    def __call__(self, infoRt, assignments):
        if len(assignments) < 12:
            return True
        if sum(np.array(list(assignments.values())) != 0) < 2:
            return False
        information = infoRt.tokenInfos
        vars = infoRt.tokens
        holdings = 0
        for t in vars:
            holdings += information[t]["totalPayment"]
        wholeB = holdings + infoRt.balance
        return infoRt.balance > 0 and (holdings / wholeB) > 0.2

In [221]:
class CorrelationComplianceConstraint(Constraint):
    def __init__(self, asset1, asset2):
        super().__init__([asset1, asset2])
    
    def __call__(self, infoRt, assignments):
        information = infoRt.tokenInfos
        a1 = self.vars[0]
        a2 = self.vars[1]
        compliance = information[a1]["corr-compliance"][a2]
        holding1 = information[a1]["totalPayment"]
        holding2 = information[a2]["totalPayment"]
        if holding1 == 0 or holding2 == 0:
            return True
        bigger = holding1 if holding1 > holding2 else holding2
        smaller = holding1 if holding1 < holding2 else holding2
        ratio = smaller / bigger
        if 0.1 < compliance <= 0.2:
            return ratio < 1/1.5
        elif 0 < compliance <= 0.1:
            return ratio < 1/2
        elif -0.1 < compliance <= 0:
            return ratio < 1/3
        elif -0.2 <= compliance <= -0.1:
            return ratio < 1/4
        elif compliance < -0.2:
            return ratio < 1/8
        return True

In [220]:
class StopLossConstraint(Constraint):
    def __init__(self, asset):
        super().__init__([asset])
    
    def __call__(self, infoRt, assignments):
        information = infoRt.tokenInfos
        a = self.vars[0]
        if information[a]["shares"] == 0:
            return True
        meanP = information[a]["totalPayment"] / information[a]["shares"]
        closeP = information[a]["close"]
        d = (closeP - meanP) / meanP
        return d >= -0.15

In [272]:
info = CryptoInfoRetriever("crypto-frames.dat", "standard_tokens_fcas.csv")
info.build()

csp = CSP()
csp.setInformationRetriever(info)
for t in info.tokens:
    csp.addVar(t)
    csp.addConstraint(StopLossConstraint(t))
for t in info.tokens:
    for tx in info.tokens:
        if t != tx:
            csp.addConstraint(CorrelationComplianceConstraint(t, tx))
csp.addConstraint(BalanceHoldingsConstraint())
csp.addConstraint(FcasConstraint())

In [160]:
def revise(csp, arc):
    revised = False
    a1 = arc.vars[0]
    a2 = arc.vars[1]
    d1 = csp.retriever.getInfo(a1)["domain"]
    d2 = csp.retriever.getInfo(a2)["domain"]
    for x in d1:
        can = False
        csp.retriever.order(a1, x)
        for y in d2:
            csp.retriever.order(a2, y)
            can = can or arc(csp.retriever)
            csp.retriever.order(a2, -y)
            if can:
                break
        csp.retriever.order(a1, -x)
        if not can:
            d1.remove(x)
            print(x)
            revised = True
    return revised

In [161]:
def ac3(csp):
    queue = list(filter(lambda x: isinstance(x, CorrelationComplianceConstraint), csp.constraints))
    while len(queue) > 0:
        arc = queue.pop(0)
        a1 = arc.vars[0]
        a2 = arc.vars[1]
        if revise(csp, arc):
            d1 = csp.retriever.getInfo(a1)["domain"]
            if len(d1) == 0:
                return False
            for t in csp.retriever.tokens:
                if t != a2:
                    for c in csp.constraints:
                        if set(c.vars) == set([t, a2]):
                            queue.append(c)
    return True

In [236]:
def backtrackingSearch(csp):
    return backtrack(csp, {})

In [268]:
def backtrack(csp, assignment):
    if len(assignment) == len(csp.vars):
        return assignment
    v = list(filter(lambda x: x not in assignment, csp.vars))[0]
    rt = csp.retriever
    for a in rt.getInfo(v)["domain"]:
        rt.order(v, a)
        consistent = True
        for c in csp.constraints:
            consistent = consistent and c(rt, assignment)
        # TODO: add Inferences
        if consistent:
            assignment[v] = a
            r = backtrack(csp, assignment)
            if r is not None:
                return r
            del assignment[v]
        rt.order(v, -a)
    return None

In [273]:
t = backtrackingSearch(csp)

In [274]:
t

{'BTS': 0.0,
 'ZRX': 0.0,
 'DOCK': 0.0,
 'ETH': 0.0,
 'ALGO': 0.0,
 'SNX': 0.0,
 'IOTX': 0.0,
 'LEND': 0.0,
 'MCO': 0.0,
 'DNT': 0.0,
 'XMR': 0.028981307056948267,
 'MKR': 0.002812702162967963,
 'VET': 730.7158323045624,
 'SC': 0.0,
 'MITH': 0.0,
 'UNI': 0.0,
 'XLM': 0.0,
 'FTT': 0.0,
 'BNB': 0.0,
 'NEAR': 0.0,
 'BLZ': 0.0,
 'WTC': 0.0,
 'COMP': 0.0,
 'ADA': 0.0,
 'DENT': 0.0,
 'NEO': 0.0,
 'KMD': 0.0,
 'BCH': 0.0,
 'KAVA': 0.0,
 'ANT': 0.0,
 'AAVE': 0.0,
 'CTXC': 0.0,
 'STORM': 0.0,
 'ZIL': 0.0,
 'ICX': 0.0,
 'KEY': 0.0,
 'FUN': 0.0,
 'NMR': 0.0,
 'XRP': 0.0,
 'DASH': 0.0,
 'EGLD': 0.0,
 'ZEN': 0.0,
 'BAL': 0.0,
 'MFT': 0.0,
 'STORJ': 0.0,
 'IOST': 0.0,
 'THETA': 0.0,
 'DOT': 0.0,
 'NPXS': 0.0,
 'ETC': 0.0,
 'TCT': 0.0,
 'BNT': 0.0,
 'TOMO': 0.0,
 'FET': 0.0,
 'TRX': 0.0,
 'ZEC': 0.0,
 'ATM': 0.0,
 'RLC': 0.0,
 'MTL': 0.0,
 'ONT': 0.0,
 'CVC': 0.0,
 'STX': 0.0,
 'YFI': 0.0,
 'QTUM': 0.0,
 'DATA': 0.0,
 'ARDR': 0.0,
 'ENJ': 0.0,
 'DOGE': 0.0,
 'KNC': 0.0,
 'NKN': 0.0,
 'MANA': 0.0,
 'M

In [ ]:
def cspT():
    # CSP Over Time
    while not csp.retriever.finished():
        t = backtrackingSearch(csp)
        csp.retriever.next()